In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data acquired

In [ ]:

df_train=pd.read_csv("/kaggle/input/predicting-random-numbers/train.csv")
df_test=pd.read_csv("/kaggle/input/predicting-random-numbers/test.csv")
df_train

In [ ]:
df_test

# Know your data

In [ ]:
df_train.info()

In [ ]:
#missing values
df_train[df_train.isnull().any(axis=1)] 

In [ ]:
print(df_train.describe(include='all'))

In [ ]:
df_train.columns

In [ ]:
unique_values = df_train['timestamp'].unique()
print(unique_values)

In [ ]:
#unique_delays = df_train['timedelay'].unique()
#print(unique_delays)

In [ ]:
uniques = df_train['name'].unique()
print(uniques)

# Clean the data for predicting number

In [ ]:
#remove the rows with NaN in columns" name or number" both.
df_train = df_train.dropna(subset=['name', 'number'])

In [ ]:
#missing values
df_train[df_train.isnull().any(axis=1)] 

In [ ]:
df_train

In [ ]:
df_train['number'].value_counts().plot(kind='bar')

In [ ]:
# Let pandas try to automatically figure out the format
# `errors='coerce'` will turn un-parsable timestamps into NaT (Not a Time)
df_train['timestamp_clean'] = pd.to_datetime(df_train['timestamp'], errors='coerce')

In [ ]:
df_train[(df_train['timestamp_clean'].isnull())]

In [ ]:
df_train['timestamp_clean']

In [ ]:
df_train.loc[df_train['name'] == 'Utpal_2301PH24']


In [ ]:
df_train.loc[df_train['name'] == 'av']


In [ ]:
df_train.loc[df_train['name'] == 'empty']


In [ ]:
import datetime
import pytz

# Your timestamp
timestamp = 1754205926

# Convert to UTC datetime
dt_utc = datetime.datetime.utcfromtimestamp(timestamp)

# Convert to your local timezone (for example, Asia/Kolkata)
local_tz = pytz.timezone('Asia/Kolkata')  # Use appropriate timezone
dt_local = dt_utc.replace(tzinfo=pytz.utc).astimezone(local_tz)

print("UTC datetime:", dt_utc)
print("Local datetime:", dt_local)


In [ ]:

import datetime
import pytz

# Your timestamp
timestamp = 1753871403	

# Convert to UTC datetime
dt_utc = datetime.datetime.utcfromtimestamp(timestamp)

# Convert to your local timezone (for example, Asia/Kolkata)
local_tz = pytz.timezone('Asia/Kolkata')  # Use appropriate timezone
dt_local = dt_utc.replace(tzinfo=pytz.utc).astimezone(local_tz)

print("UTC datetime:", dt_utc)
print("Local datetime:", dt_local)

In [ ]:
import datetime
import pytz

local_tz = pytz.timezone('Asia/Kolkata')

def convert_to_local(ts_str):
    ts = int(ts_str)
    dt_utc = datetime.datetime.utcfromtimestamp(ts)
    dt_local = dt_utc.replace(tzinfo=pytz.utc).astimezone(local_tz)
    return dt_local.strftime('%Y-%m-%d %H:%M:%S')



pattern = r'^\d{10}$'

# Create a boolean mask safely by filling NaNs with empty string first
mask = df_train['timestamp'].fillna('').str.match(pattern)

# Use the mask to create/assign new column after conversion
df_train.loc[mask, 'timestamp_clean'] = df_train.loc[mask, 'timestamp'].apply(convert_to_local)

print(df_train)


In [ ]:
df_train[(df_train['timestamp_clean'].isnull())]

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train['timedelay'] = pd.to_numeric(df_train['timedelay'], errors='coerce')


In [ ]:
min_delay = df_train['timedelay'].min()
max_delay = df_train['timedelay'].max()
print(f"Timedelay ranges from {min_delay} to {max_delay}")


In [ ]:
df_train.loc[(df_train['timedelay'] >= 100)]



In [ ]:
df_train.drop(df_train[df_train['timedelay'] >= 100].index, inplace=True)


In [ ]:
df_train

In [ ]:
df_train['number'] = pd.to_numeric(df_train['number'], errors='coerce')


In [ ]:
print(df_train.describe(include='all'))

In [ ]:
df_train

In [ ]:
# Find names that appear exactly once
unique_names = df_train['name'].value_counts()[df_train['name'].value_counts() == 1].index

# Replace those unique names with 'anonymous'
df_train.loc[df_train['name'].isin(unique_names), 'name'] = 'anonymous'


In [ ]:
df_train

In [ ]:
df_train.loc[df_train['name'] == 'anonymous']


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import datetime
import pytz
import sys

# =============================================================================
# STEP 1 & 2: LOAD DATA AND LEARN PARAMETERS
# =============================================================================
print("Step 1 & 2: Loading data and learning parameters...")
try:
    df_train_raw = pd.read_csv("/kaggle/input/predicting-random-numbers/train.csv")
    df_test_raw = pd.read_csv("/kaggle/input/predicting-random-numbers/test.csv")
except FileNotFoundError as e:
    sys.exit(f"ERROR: File not found. Please check the path: {e.filename}")

median_timedelay = pd.to_numeric(df_train_raw['timedelay'], errors='coerce').median()
mode_timestamp = pd.to_datetime(df_train_raw['timestamp'], errors='coerce').mode()[0]
name_freq_map = df_train_raw['name'].value_counts().to_dict()
unique_names_to_replace = df_train_raw['name'].value_counts()[df_train_raw['name'].value_counts() == 1].index

# =============================================================================
# STEP 3: MASTER PROCESSING FUNCTION
# =============================================================================
print("Step 3: Building and applying the master processing function...")

def convert_to_local(ts_str):
    local_tz = pytz.timezone('Asia/Kolkata')
    ts = int(ts_str)
    dt_utc = datetime.datetime.utcfromtimestamp(ts)
    dt_local = dt_utc.replace(tzinfo=pytz.utc).astimezone(local_tz)
    return dt_local.replace(tzinfo=None)

def process_data(df, unique_names_rules, median_delay_rule, mode_ts_rule, name_freq_rules):
    df_processed = df.copy()
    df_processed['name'] = df_processed['name'].fillna('anonymous')
    df_processed.loc[df_processed['name'].isin(unique_names_rules), 'name'] = 'anonymous'
    df_processed['timestamp_clean'] = pd.to_datetime(df_processed['timestamp'], errors='coerce')
    pattern = r'^\d{10}$'
    mask = df_processed['timestamp'].fillna('').str.match(pattern)
    df_processed.loc[mask, 'timestamp_clean'] = df_processed.loc[mask, 'timestamp'].apply(convert_to_local)
    df_processed['timestamp_clean'] = df_processed['timestamp_clean'].fillna(mode_ts_rule)
    df_processed['timedelay'] = pd.to_numeric(df_processed['timedelay'], errors='coerce')
    df_processed['timedelay'] = df_processed['timedelay'].fillna(median_delay_rule)
    df_processed['hour'] = df_processed['timestamp_clean'].dt.hour
    df_processed['day_of_week'] = df_processed['timestamp_clean'].dt.dayofweek
    df_processed['hour_sin'] = np.sin(2 * np.pi * df_processed['hour']/24)
    df_processed['hour_cos'] = np.cos(2 * np.pi * df_processed['hour']/24)
    df_processed['timedelay_log'] = np.log1p(df_processed['timedelay'])
    df_processed['name_freq'] = df_processed['name'].map(name_freq_rules).fillna(1)
    df_processed['is_anonymous'] = (df_processed['name'].str.lower() == 'anonymous').astype(int)
    return df_processed

train_df = process_data(df_train_raw, unique_names_to_replace, median_timedelay, mode_timestamp, name_freq_map)
test_df = process_data(df_test_raw, unique_names_to_replace, median_timedelay, mode_timestamp, name_freq_map)

# =============================================================================
# STEP 4: CLEAN TARGET AND APPLY FILTERS
# =============================================================================
print("Step 4: Cleaning target variable and filtering training data...")

train_df['number'] = pd.to_numeric(df_train_raw['number'], errors='coerce')
train_df.dropna(subset=['number'], inplace=True)
train_df['number'] = train_df['number'].astype(int)
train_df = train_df[train_df['timedelay'] < 100].copy()

# =============================================================================
# STEP 5: CREATE ADVANCED FEATURES (WITH ROBUST SEPARATION)
# =============================================================================
print("Step 5: Creating advanced features on clean data...")

# CORRECTED LOGIC: Add a 'source' column to tag the data BEFORE combining
train_df['source'] = 'train'
test_df['source'] = 'test'

combined_df = pd.concat([train_df, test_df], sort=False, ignore_index=True)
combined_df.sort_values(['name', 'timestamp_clean'], inplace=True)

combined_df['lag1_number'] = combined_df.groupby('name')['number'].shift(1)
agg_stats = combined_df.groupby('name')['timedelay_log'].agg(['mean', 'std'])
agg_stats.columns = ['user_mean_delay', 'user_std_delay']
combined_df = combined_df.merge(agg_stats, on='name', how='left')
combined_df['delay_x_hour'] = combined_df['timedelay_log'] * combined_df['hour']
feature_cols_to_fill = ['lag1_number', 'user_mean_delay', 'user_std_delay']
for col in feature_cols_to_fill:
    combined_df[col] = combined_df[col].fillna(-1)

# CORRECTED LOGIC: Separate using the 'source' tag for 100% accuracy
train_df_final = combined_df[combined_df['source'] == 'train'].drop(columns=['source'])
test_df_final = combined_df[combined_df['source'] == 'test'].drop(columns=['source'])

# =============================================================================
# STEP 6: MODEL TRAINING AND SUBMISSION
# =============================================================================
print("Step 6: Starting model training and submission...")

features_to_use = [
    'hour_sin', 'hour_cos', 'timedelay_log', 'name_freq',
    'is_anonymous', 'delay_x_hour', 'lag1_number',
    'user_mean_delay', 'user_std_delay'
]

X = train_df_final[features_to_use]
y = train_df_final['number'].astype(int) # Ensure y is int
X_test = test_df_final[features_to_use]

print("\nStarting 5-Fold Cross-Validation...")
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_f1_scores = []
for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    model = lgb.LGBMClassifier(objective='multiclass', num_class=10, class_weight='balanced', random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    f1 = f1_score(y_val, preds, average='macro')
    fold_f1_scores.append(f1)
    print(f"Fold {fold+1} Macro F1-Score: {f1:.4f}")
print(f"\nMean Macro F1-Score across 5 folds: {np.mean(fold_f1_scores):.4f}")

print("\nTraining final model...")
final_model = lgb.LGBMClassifier(objective='multiclass', num_class=10, class_weight='balanced', random_state=42)
final_model.fit(X, y)
test_predictions = final_model.predict(X_test)

submission_df = pd.DataFrame({'id': test_df_final['id'], 'number': test_predictions})
submission_df.to_csv('submission.csv', index=False)

print("\nBhai, it is done! Congratulations!")
print(submission_df.head())

In [ ]:
submission_df

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import datetime
import pytz
import sys

# =============================================================================
# STEP 1: LOAD YOUR ORIGINAL RAW DATA
# =============================================================================
print("Step 1: Loading original raw data...")
try:
    df_train_raw = pd.read_csv("/kaggle/input/predicting-random-numbers/train.csv")
    df_test_raw = pd.read_csv("/kaggle/input/predicting-random-numbers/test.csv")
except FileNotFoundError as e:
    sys.exit(f"ERROR: File not found. Please check the path: {e.filename}")

# =============================================================================
# STEP 2: LEARN PARAMETERS FROM TRAINING DATA ONLY
# =============================================================================
print("Step 2: Learning imputation parameters from training data...")
median_timedelay = pd.to_numeric(df_train_raw['timedelay'], errors='coerce').median()
mode_timestamp = pd.to_datetime(df_train_raw['timestamp'], errors='coerce').mode()[0]
name_freq_map = df_train_raw['name'].value_counts().to_dict()
unique_names_to_replace = df_train_raw['name'].value_counts()[df_train_raw['name'].value_counts() == 1].index

# =============================================================================
# STEP 3: CREATE THE MASTER PROCESSING FUNCTION
# =============================================================================
print("Step 3: Building and applying the master processing function...")

def convert_to_local(ts_str):
    local_tz = pytz.timezone('Asia/Kolkata')
    ts = int(ts_str)
    dt_utc = datetime.datetime.utcfromtimestamp(ts)
    dt_local = dt_utc.replace(tzinfo=pytz.utc).astimezone(local_tz)
    return dt_local.replace(tzinfo=None)

def process_data(df):
    df_processed = df.copy()
    df_processed['name'] = df_processed['name'].fillna('anonymous')
    df_processed.loc[df_processed['name'].isin(unique_names_to_replace), 'name'] = 'anonymous'
    df_processed['timestamp_clean'] = pd.to_datetime(df_processed['timestamp'], errors='coerce')
    pattern = r'^\d{10}$'
    mask = df_processed['timestamp'].fillna('').str.match(pattern)
    df_processed.loc[mask, 'timestamp_clean'] = df_processed.loc[mask, 'timestamp'].apply(convert_to_local)
    df_processed['timestamp_clean'] = df_processed['timestamp_clean'].fillna(mode_timestamp)
    df_processed['timedelay'] = pd.to_numeric(df_processed['timedelay'], errors='coerce')
    df_processed['timedelay'] = df_processed['timedelay'].fillna(median_timedelay)
    df_processed['hour'] = df_processed['timestamp_clean'].dt.hour
    df_processed['day_of_week'] = df_processed['timestamp_clean'].dt.dayofweek
    df_processed['hour_sin'] = np.sin(2 * np.pi * df_processed['hour']/24)
    df_processed['hour_cos'] = np.cos(2 * np.pi * df_processed['hour']/24)
    df_processed['timedelay_log'] = np.log1p(df_processed['timedelay'])
    df_processed['name_freq'] = df_processed['name'].map(name_freq_map).fillna(1)
    df_processed['is_anonymous'] = (df_processed['name'].str.lower() == 'anonymous').astype(int)
    return df_processed

train_df = process_data(df_train_raw)
test_df = process_data(df_test_raw)

# =============================================================================
# STEP 4: CLEAN TARGET AND APPLY FILTERS
# =============================================================================
print("Step 4: Cleaning target variable and filtering training data...")
train_df['number'] = pd.to_numeric(train_df['number'], errors='coerce')
train_df.dropna(subset=['number'], inplace=True)
train_df['number'] = train_df['number'].astype(int)
train_df = train_df[train_df['timedelay'] < 100].copy()

# =============================================================================
# STEP 5: CREATE ADVANCED FEATURES
# =============================================================================
print("Step 5: Creating advanced features...")
train_df['source'] = 'train'
test_df['source'] = 'test'
combined_df = pd.concat([train_df, test_df], sort=False, ignore_index=True)
combined_df.sort_values(['name', 'timestamp_clean'], inplace=True)
combined_df['lag1_number'] = combined_df.groupby('name')['number'].shift(1)
agg_stats = combined_df.groupby('name')['timedelay_log'].agg(['mean', 'std'])
agg_stats.columns = ['user_mean_delay', 'user_std_delay']
combined_df = combined_df.merge(agg_stats, on='name', how='left')
combined_df['delay_x_hour'] = combined_df['timedelay_log'] * combined_df['hour']
feature_cols_to_fill = ['lag1_number', 'user_mean_delay', 'user_std_delay']
for col in feature_cols_to_fill:
    combined_df[col] = combined_df[col].fillna(-1)

train_df_final = combined_df[combined_df['source'] == 'train'].drop(columns=['source'])
test_df_final = combined_df[combined_df['source'] == 'test'].drop(columns=['source'])

# =============================================================================
# STEP 6: MODEL TRAINING AND SUBMISSION ("PARANOID MODE")
# =============================================================================
print("Step 6: Starting model training and submission...")
features_to_use = [
    'hour_sin', 'hour_cos', 'timedelay_log', 'name_freq',
    'is_anonymous', 'delay_x_hour', 'lag1_number',
    'user_mean_delay', 'user_std_delay'
]
X = train_df_final[features_to_use]
y = train_df_final['number'].astype(int)
X_test = test_df_final[features_to_use]

print("\nStarting 5-Fold Cross-Validation...")
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_f1_scores = []
for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    model = lgb.LGBMClassifier(objective='multiclass', num_class=10, class_weight='balanced', random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    f1 = f1_score(y_val, preds, average='macro')
    fold_f1_scores.append(f1)
print(f"\nMean Macro F1-Score across 5 folds: {np.mean(fold_f1_scores):.4f}")

print("\nTraining final model...")
final_model = lgb.LGBMClassifier(objective='multiclass', num_class=10, class_weight='balanced', random_state=42)
final_model.fit(X, y)
test_predictions = final_model.predict(X_test)

print("\nCreating the submission file with guaranteed integer types...")
submission_df = pd.DataFrame({'id': test_df_final['id'], 'number': test_predictions})
# SAFETY CHECK 1: Force the 'number' column to be an integer.
submission_df['number'] = submission_df['number'].astype(int)
# SAFETY CHECK 2: Force the 'id' column to be an integer.
submission_df['id'] = submission_df['id'].astype(int)
submission_df.to_csv('submission.csv', index=False)

print("\nBhai, it is done! Congratulations!")
print("This submission file has its data types double-checked.")
print(submission_df.head())
print("\nFinal data types of submission file:")
print(submission_df.dtypes)

In [ ]:
submission_df

In [ ]:
test_df

In [ ]:
import pandas as pd
import sys

print("--- RUNNING SCRIPT #1: THE SAFE BASELINE MODEL ---")

# =============================================================================
# LOAD DATA
# =============================================================================
try:
    df_train_raw = pd.read_csv("/kaggle/input/predicting-random-numbers/train.csv")
    df_test_raw = pd.read_csv("/kaggle/input/predicting-random-numbers/test.csv")
except FileNotFoundError:
    sys.exit("ERROR: Could not find the Kaggle input files. Please check the path.")

# =============================================================================
# BASELINE LOGIC
# =============================================================================
# Clean the target variable to find the true mode
df_train_raw['number'] = pd.to_numeric(df_train_raw['number'], errors='coerce')
df_train_raw.dropna(subset=['number'], inplace=True)
df_train_raw['number'] = df_train_raw['number'].astype(int)

# Find the single most common number
most_frequent_number = df_train_raw['number'].mode()[0]
print(f"The most frequent number in the training data is: {most_frequent_number}")

# Create a submission dataframe
submission_df = pd.DataFrame({'id': df_test_raw['id'], 'number': most_frequent_number})

# Force the columns to be integers
submission_df['id'] = submission_df['id'].astype(int)
submission_df['number'] = submission_df['number'].astype(int)

# =============================================================================
# SAVE SUBMISSION
# =============================================================================
submission_df.to_csv('baseline_submission.csv', index=False)
print("\n'baseline_submission.csv' has been created.")
print("This is your safe submission.")
print(submission_df.head())

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import datetime
import pytz
import sys

# =============================================================================
# STEPS 1-4 (These are correct and remain the same)
# =============================================================================
print("Steps 1-4: Loading, Learning, Processing, and Cleaning...")
try:
    df_train_raw = pd.read_csv("/kaggle/input/predicting-random-numbers/train.csv")
    df_test_raw = pd.read_csv("/kaggle/input/predicting-random-numbers/test.csv")
except FileNotFoundError as e:
    sys.exit(f"ERROR: File not found. Please check the path: {e.filename}")

median_timedelay = pd.to_numeric(df_train_raw['timedelay'], errors='coerce').median()
mode_timestamp = pd.to_datetime(df_train_raw['timestamp'], errors='coerce').mode()[0]
name_freq_map = df_train_raw['name'].value_counts().to_dict()
unique_names_to_replace = df_train_raw['name'].value_counts()[df_train_raw['name'].value_counts() == 1].index

def convert_to_local(ts_str):
    local_tz = pytz.timezone('Asia/Kolkata')
    ts = int(ts_str)
    dt_utc = datetime.datetime.utcfromtimestamp(ts)
    dt_local = dt_utc.replace(tzinfo=pytz.utc).astimezone(local_tz)
    return dt_local.replace(tzinfo=None)

def process_data(df):
    df_processed = df.copy()
    df_processed['name'] = df_processed['name'].fillna('anonymous')
    df_processed.loc[df_processed['name'].isin(unique_names_to_replace), 'name'] = 'anonymous'
    df_processed['timestamp_clean'] = pd.to_datetime(df_processed['timestamp'], errors='coerce')
    pattern = r'^\d{10}$'
    mask = df_processed['timestamp'].fillna('').str.match(pattern)
    df_processed.loc[mask, 'timestamp_clean'] = df_processed.loc[mask, 'timestamp'].apply(convert_to_local)
    df_processed['timestamp_clean'] = df_processed['timestamp_clean'].fillna(mode_timestamp)
    df_processed['timedelay'] = pd.to_numeric(df_processed['timedelay'], errors='coerce')
    df_processed['timedelay'] = df_processed['timedelay'].fillna(median_timedelay)
    df_processed['hour'] = df_processed['timestamp_clean'].dt.hour
    df_processed['day_of_week'] = df_processed['timestamp_clean'].dt.dayofweek
    df_processed['hour_sin'] = np.sin(2 * np.pi * df_processed['hour']/24)
    df_processed['hour_cos'] = np.cos(2 * np.pi * df_processed['hour']/24)
    df_processed['timedelay_log'] = np.log1p(df_processed['timedelay'])
    df_processed['name_freq'] = df_processed['name'].map(name_freq_map).fillna(1)
    df_processed['is_anonymous'] = (df_processed['name'].str.lower() == 'anonymous').astype(int)
    return df_processed

train_df = process_data(df_train_raw)
test_df = process_data(df_test_raw)

train_df['number'] = pd.to_numeric(df_train_raw['number'], errors='coerce')
train_df.dropna(subset=['number'], inplace=True)
train_df['number'] = train_df['number'].astype(int)
train_df = train_df[train_df['timedelay'] < 100].copy()

# =============================================================================
# STEP 5: CREATE ADVANCED FEATURES (WITH ROBUST SEPARATION)
# =============================================================================
print("Step 5: Creating advanced features...")
train_df['source'] = 'train'
test_df['source'] = 'test'
combined_df = pd.concat([train_df, test_df], sort=False, ignore_index=True)
combined_df.sort_values(['name', 'timestamp_clean'], inplace=True)
combined_df['lag1_number'] = combined_df.groupby('name')['number'].shift(1)
agg_stats = combined_df.groupby('name')['timedelay_log'].agg(['mean', 'std'])
agg_stats.columns = ['user_mean_delay', 'user_std_delay']
combined_df = combined_df.merge(agg_stats, on='name', how='left')
combined_df['delay_x_hour'] = combined_df['timedelay_log'] * combined_df['hour']
feature_cols_to_fill = ['lag1_number', 'user_mean_delay', 'user_std_delay']
for col in feature_cols_to_fill:
    combined_df[col] = combined_df[col].fillna(-1)

train_df_final = combined_df[combined_df['source'] == 'train'].drop(columns=['source'])
test_df_final = combined_df[combined_df['source'] == 'test'].drop(columns=['source'])

# =============================================================================
# STEP 6: MODEL TRAINING AND SUBMISSION
# =============================================================================
print("Step 6: Starting model training and submission...")

# --- NEW FINAL SAFETY CHECK ---
# Remove any rows from the final test set that have a missing ID
test_df_final.dropna(subset=['id'], inplace=True)

features_to_use = [
    'hour_sin', 'hour_cos', 'timedelay_log', 'name_freq',
    'is_anonymous', 'delay_x_hour', 'lag1_number',
    'user_mean_delay', 'user_std_delay'
]
X = train_df_final[features_to_use]
y = train_df_final['number'].astype(int)
X_test = test_df_final[features_to_use] # X_test is now created from the clean test_df_final

# --- Final Model Training ---
print("\nTraining final model...")
final_model = lgb.LGBMClassifier(objective='multiclass', num_class=10, class_weight='balanced', random_state=42)
final_model.fit(X, y)
test_predictions = final_model.predict(X_test)

# --- APE Safety Fix ---
most_frequent_num = train_df_final['number'].mode()[0]
predictions_series = pd.Series(test_predictions, index=X_test.index) # Ensure index alignment
predictions_series = predictions_series.replace(0, most_frequent_num)

# --- Create Final Submission ---
# The 'id' column is now guaranteed to be clean
submission_df = pd.DataFrame({'id': test_df_final['id'], 'number': predictions_series})
submission_df['id'] = submission_df['id'].astype(int)
submission_df['number'] = submission_df['number'].astype(int)

submission_df.to_csv('advanced_submission.csv', index=False)
print("\n'advanced_submission.csv' has been created.")
print("This is your best shot submission.")
print(submission_df.head())